<a href="https://colab.research.google.com/github/rite2rio/AI-ML-Portfolio/blob/main/ReflectionPattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai colorama

The first pattern we are going to implement is the **reflection pattern**.


<img src="https://drive.google.com/uc?id=1tXchMzG0_TjGGM1lOCULztwRZj7UrBz-" alt="Alt text" width="600"/>


This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

In this example, we'll test the Python coding skills of GPT-4o (remember that's the LLM we are going to use for all the tutorials). In particular, we are going to ask the LLM to code a famous sorting algorithm: **Merge Sort**.

---

<img src="https://drive.google.com/uc?id=1QamQqbPOCYizJ8lcc_4MF3u1FCNPTUpW" alt="Alt text" width="400"/>

In [3]:
import os
from pprint import pprint
from IPython.display import display_markdown
from google.colab import userdata

from openai import OpenAI

# Remember to add your OpenAI API Key to the user data
client = OpenAI(
    api_key=userdata.get('OpenAI_API_Key')
)
model = "gpt-4o"

## Generation Step

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.

In [4]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [6]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the merge sort implementation.

In [7]:
generation_chat_history

[{'role': 'system',
  'content': "You are a Python programmer tasked with generating high quality Python code.Your task is to Generate the best content possible for the user's request. If the user provides critique,respond with a revised version of your previous attempt."},
 {'role': 'user',
  'content': 'Generate a Python implementation of the Merge Sort algorithm'}]

In [8]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model=model
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [9]:
display_markdown(mergesort_code, raw=True)

Certainly! Here is a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    # Divide the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half
    left_sorted = merge_sort(left_half)
    right_sorted = merge_sort(right_half)

    # Merge the two sorted halves
    return merge(left_sorted, right_sorted)

def merge(left, right):
    sorted_arr = []
    i = j = 0

    # Merge the two halves into a sorted array
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            sorted_arr.append(left[i])
            i += 1
        else:
            sorted_arr.append(right[j])
            j += 1

    # Append any remaining elements from the left half
    while i < len(left):
        sorted_arr.append(left[i])
        i += 1

    # Append any remaining elements from the right half
    while j < len(right):
        sorted_arr.append(right[j])
        j += 1

    return sorted_arr

# Example usage:
data = [38, 27, 43, 3, 9, 82, 10]
sorted_data = merge_sort(data)
print("Sorted array:", sorted_data)
```

### Explanation:

1. **Divide:** The `merge_sort` function is a recursive function that splits the array into halves until the base case is reached (an array with a single element or empty), where it simply returns the array.

2. **Conquer:** The two halves are individually sorted by recursive calls to `merge_sort`.

3. **Merge:** The `merge` function takes two sorted arrays and merges them into a single sorted array by comparing the elements of the two arrays and placing the smallest into the resulting array.

This implementation efficiently sorts the array and demonstrates the classic divide-and-conquer strategy that makes merge sort a stable and very effective sorting algorithm, especially for large datasets.

## Reflection step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

In [10]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [11]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [12]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model=model
).choices[0].message.content

In [13]:
display_markdown(critique, raw=True)

Your implementation of the Merge Sort algorithm is correct and well-structured. It follows the classic divide-and-conquer approach, and the code is clean and easy to understand. Here are some critiques and recommendations for potential improvements:

1. **In-place Merging (Optional):**
   - Your current implementation creates new lists during the merge process, which can consume additional memory. Although this aligns with the standard merge sort where extra space is used, you can explore in-place sorting techniques to reduce space complexity at the cost of increased algorithm complexity.

2. **Naming Convention:**
   - Your naming conventions are clear, but you might consider even more descriptive names, such as `merge_sort_helper` instead of `merge` to differentiate it more from the sorting function itself.

3. **Edge Cases:**
   - While you've handled arrays with zero or one element nicely, ensure that the implementation behaves correctly with edge cases like arrays where all elements are the same, or arrays that are already sorted.
  
4. **Performance Measurement:**
   - For practical applications, consider adding a timing mechanism around the calls to `merge_sort` to measure performance on real datasets, which could provide insights into efficiency.
  
5. **Documentation:**
   - Even though the code is relatively self-explanatory, adding docstrings to your functions would improve readability and maintainability, especially in larger projects. These should explain what each function does, its parameters, and what it returns.

6. **Unit Tests:**
   - Implement unit tests to verify your merge sort on various input scenarios. This helps ensure that bugs are identified and fixed before deployment or integration into larger systems.

Here's a small example of how you might add a docstring to the `merge_sort` function:

```python
def merge_sort(arr):
    """
    Sorts an array using the merge sort algorithm.

    Args:
        arr (list of int): A list of integers to sort.

    Returns:
        list of int: A new list containing the sorted integers.
    """
```

Overall, this is a solid implementation of merge sort. Considering these recommendations could enhance the robustness and usability of your code, especially if incorporated into a larger codebase.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [14]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Next Steps

In [16]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model=model
).choices[0].message.content

In [17]:
display_markdown(essay, raw=True)

Thank you for the insightful feedback. Let's implement some of these suggestions to improve the merge sort code.

Here's an updated version that incorporates documentation, unit testing, and more descriptive function naming. I'll keep the in-place merge as a potential enhancement for later due to its complexity.

```python
def merge_sort(arr):
    """
    Sorts an array using the merge sort algorithm.

    Args:
        arr (list): A list of elements to sort.

    Returns:
        list: A new list containing the sorted elements.
    """
    if len(arr) <= 1:
        return arr

    # Divide the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort each half
    left_sorted = merge_sort(left_half)
    right_sorted = merge_sort(right_half)

    # Merge the two sorted halves
    return merge_sorted_halves(left_sorted, right_sorted)


def merge_sorted_halves(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    Args:
        left (list): The first sorted list.
        right (list): The second sorted list.

    Returns:
        list: A new sorted list containing all elements from both input lists.
    """
    sorted_arr = []
    i = j = 0

    # Merge the two halves into a sorted array
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            sorted_arr.append(left[i])
            i += 1
        else:
            sorted_arr.append(right[j])
            j += 1

    # Append any remaining elements from the left half
    sorted_arr.extend(left[i:])

    # Append any remaining elements from the right half
    sorted_arr.extend(right[j:])

    return sorted_arr


def test_merge_sort():
    assert merge_sort([]) == []
    assert merge_sort([1]) == [1]
    assert merge_sort([2, 1]) == [1, 2]
    assert merge_sort([5, 3, 8, 6, 2]) == [2, 3, 5, 6, 8]
    assert merge_sort([7, 2, 5, 3, 7, 0, 9]) == [0, 2, 3, 5, 7, 7, 9]
    assert merge_sort([8, 8, 8, 8]) == [8, 8, 8, 8]
    print("All tests passed.")

# Example usage and testing
if __name__ == "__main__":
    data = [38, 27, 43, 3, 9, 82, 10]
    sorted_data = merge_sort(data)
    print("Sorted array:", sorted_data)

    # Run tests
    test_merge_sort()
```

### Enhancements:

1. **Docstrings**: Added detailed docstrings to describe the purpose, parameters, and return values of functions.
  
2. **Descriptive Naming**: Changed the `merge` function to `merge_sorted_halves` for clarity.

3. **Tests**: Added a simple `test_merge_sort` function to run unit tests on several edge cases and validate the correctness of the sorting algorithm.

These refinements make the code clearer and more robust, aiding in maintainability and future development.

And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Building a Reflection Agent

In [18]:
# @title
"""
This is a collection of helper functions and methods we are going to use in
the Agent implementation. You don't need to know the specific implementation
of these to follow the Agent code. But, if you are curious, feel free to check
them out.
"""

import time

from colorama import Fore
from colorama import Style

# calling the completions function
def completions_create(client, messages: list, model: str) -> str:
    """
    Sends a request to the client's `completions.create` method to interact with the language model.

    Args:
        client (OpenAI): The OpenAI client object
        messages (list[dict]): A list of message objects containing chat history for the model.
        model (str): The model to use for generating tool calls and responses.

    Returns:
        str: The content of the model's response.
    """
    response = client.chat.completions.create(messages=messages, model=model)
    return str(response.choices[0].message.content)

# creates a prompt structure
def build_prompt_structure(prompt: str, role: str, tag: str = "") -> dict:
    """
    Builds a structured prompt that includes the role and content.

    Args:
        prompt (str): The actual content of the prompt.
        role (str): The role of the speaker (e.g., user, assistant).

    Returns:
        dict: A dictionary representing the structured prompt.
    """
    if tag:
        prompt = f"<{tag}>{prompt}</{tag}>"
    return {"role": role, "content": prompt}

# append the chat history so that the LLM is aware of the previous
# conversation or question by the user and the reflection history as well
def update_chat_history(history: list, msg: str, role: str):
    """
    Updates the chat history by appending the latest response.

    Args:
        history (list): The list representing the current chat history.
        msg (str): The message to append.
        role (str): The role type (e.g. 'user', 'assistant', 'system')
    """
    history.append(build_prompt_structure(prompt=msg, role=role))


class ChatHistory(list):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """Initialise the queue with a fixed total length.

        Args:
            messages (list | None): A list of initial messages
            total_length (int): The maximum number of messages the chat history can hold.
        """
        if messages is None:
            messages = []

        super().__init__(messages)
        self.total_length = total_length

    def append(self, msg: str):
        """Add a message to the queue.

        Args:
            msg (str): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(0)
        super().append(msg)



class FixedFirstChatHistory(ChatHistory):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """Initialise the queue with a fixed total length.

        Args:
            messages (list | None): A list of initial messages
            total_length (int): The maximum number of messages the chat history can hold.
        """
        super().__init__(messages, total_length)

    def append(self, msg: str):
        """Add a message to the queue. The first messaage will always stay fixed.

        Args:
            msg (str): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(1)
        super().append(msg)

# to pring logs in different colors for generation and reflection prompts
def fancy_print(message: str) -> None:
    """
    Displays a fancy print message.

    Args:
        message (str): The message to display.
    """
    print(Style.BRIGHT + Fore.CYAN + f"\n{'=' * 50}")
    print(Fore.MAGENTA + f"{message}")
    print(Style.BRIGHT + Fore.CYAN + f"{'=' * 50}\n")
    time.sleep(0.5)

# will track how many steps the reflection api is taking
def fancy_step_tracker(step: int, total_steps: int) -> None:
    """
    Displays a fancy step tracker for each iteration of the generation-reflection loop.

    Args:
        step (int): The current step in the loop.
        total_steps (int): The total number of steps in the loop.
    """
    fancy_print(f"STEP {step + 1}/{total_steps}")

In [22]:
# system prompt
BASE_GENERATION_SYSTEM_PROMPT = """
Your task is to Generate the best content possible for the user's request.
If the user provides critique, respond with a revised version of your previous attempt.
You must always output the revised content.
"""
# reflection prompt
BASE_REFLECTION_SYSTEM_PROMPT = """
You are tasked with generating critique and recommendations to the user's generated content.
If the user content has something wrong or something to be improved, output a list of recommendations
and critiques.
"""

# the reflection agent to critique on the reflections
class ReflectionAgent:
    """
    A class that implements a Reflection Agent, which generates responses and reflects
    on them using the LLM to iteratively improve the interaction. The agent first generates
    responses based on provided prompts and then critiques them in a reflection step.

    Attributes:
        model (str): The model name used for generating and reflecting on responses.
        client (OpenAI): An instance of the OpenAI client to interact with the language model.
    """

    def __init__(self, model: str = "gpt-4o"):
        self.client = OpenAI(
          api_key=userdata.get('OpenAI_API_Key')
        )
        self.model = model

    def _request_completion(
        self,
        history: list,
        verbose: int = 0,
        log_title: str = "COMPLETION",
        log_color: str = "",
    ):
        """
        A private method to request a completion from the OpenAI model.

        Args:
            history (list): A list of messages forming the conversation or reflection history.
            verbose (int, optional): The verbosity level. Defaults to 0 (no output).

        Returns:
            str: The model-generated response.
        """
        output = completions_create(self.client, history, self.model)

        if verbose > 0:
            print(log_color, f"\n\n{log_title}\n\n", output)

        return output

# response generation
    def generate(self, generation_history: list, verbose: int = 0) -> str:
        """
        Generates a response based on the provided generation history using the model.

        Args:
            generation_history (list): A list of messages forming the conversation or generation history.
            verbose (int, optional): The verbosity level, controlling printed output. Defaults to 0.

        Returns:
            str: The generated response.
        """
        return self._request_completion(
            generation_history, verbose, log_title="GENERATION", log_color=Fore.BLUE
        )

    def reflect(self, reflection_history: list, verbose: int = 0) -> str:
        """
        Reflects on the generation history by generating a critique or feedback.

        Args:
            reflection_history (list): A list of messages forming the reflection history, typically based on
                                       the previous generation or interaction.
            verbose (int, optional): The verbosity level, controlling printed output. Defaults to 0.

        Returns:
            str: The critique or reflection response from the model.
        """
        return self._request_completion(
            reflection_history, verbose, log_title="REFLECTION", log_color=Fore.GREEN
        )


    def run(
        self,
        user_msg: str, # user message or query
        generation_system_prompt: str = "", # generation
        reflection_system_prompt: str = "", # reflextion prompt
        n_steps: int = 10, # maximum steps for generate- reflection the agent should take , it defulats to 3 as provided in the prompt below
        verbose: int = 0,
    ) -> str:
        """
        Runs the ReflectionAgent over multiple steps, alternating between generating a response
        and reflecting on it for the specified number of steps.

        Args:
            user_msg (str): The user message or query that initiates the interaction.
            generation_system_prompt (str, optional): The system prompt for guiding the generation process.
            reflection_system_prompt (str, optional): The system prompt for guiding the reflection process.
            n_steps (int, optional): The number of generate-reflect cycles to perform. Defaults to 3.
            verbose (int, optional): The verbosity level controlling printed output. Defaults to 0.

        Returns:
            str: The final generated response after all cycles are completed.
        """
        generation_system_prompt += BASE_GENERATION_SYSTEM_PROMPT
        reflection_system_prompt += BASE_REFLECTION_SYSTEM_PROMPT

        # Given the iterative nature of the Reflection Pattern, we might exhaust the LLM context (or
        # make it really slow). That's the reason I'm limitting the chat history to three messages.
        # The `FixedFirstChatHistory` is a very simple class, that creates a Queue that always keeps
         # ( it always keeps the system prompt fixed , and the other 2 messages will change)
        # fixed the first message. I thought this would be useful for maintaining the system prompt
        # in the chat history.
        generation_history = FixedFirstChatHistory(
            [
                build_prompt_structure(prompt=generation_system_prompt, role="system"),
                build_prompt_structure(prompt=user_msg, role="user"),
            ],
            total_length=3,
        )

        reflection_history = FixedFirstChatHistory(
            [build_prompt_structure(prompt=reflection_system_prompt, role="system")],
            total_length=3,
        )

# iterate through number of steps
        for step in range(n_steps):
            if verbose > 0:
                fancy_step_tracker(step, n_steps)

            # Step 1 Generate the response
            generation = self.generate(generation_history, verbose=verbose)
            update_chat_history(generation_history, generation, "assistant")
            update_chat_history(reflection_history, generation, "user")

            # Reflect and critique the generation
            critique = self.reflect(reflection_history, verbose=verbose)

            update_chat_history(generation_history, critique, "user")
            update_chat_history(reflection_history, critique, "assistant")

        return generation


In [23]:
agent = ReflectionAgent() # instantiated the reflection agent

In [24]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [25]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=3,
    verbose=1,
)


STEP 1/3

 

GENERATION

 Certainly! Here's an implementation of the Merge Sort algorithm in Python:

```python
def merge_sort(arr):
    """
    Sorts an array using the merge sort algorithm.

    Parameters:
    arr (list): The list to be sorted.

    Returns:
    list: A new sorted list.
    """
    if len(arr) <= 1:
        return arr

    # Find the middle point and divide the array into two halves
    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort the two halves
    sorted_left = merge_sort(left_half)
    sorted_right = merge_sort(right_half)

    # Merge the sorted halves
    return merge(sorted_left, sorted_right)

def merge(left, right):
    """
    Merges two sorted lists into one sorted list.

    Parameters:
    left (list): The first sorted list.
    right (list): The second sorted list.

    Returns:
    list: A merged and sorted list.
    """
    sorted_list = []
    left_index, right_index = 0, 0

    # Compare each elem

In [26]:
print(final_response)

Thank you for the detailed feedback! I’ve incorporated your suggestions into the code to address all the mentioned points. Here's the updated version:

```python
from typing import List

def merge_sort(arr: List[int]) -> List[int]:
    """
    Sorts an array of integers using the merge sort algorithm.

    This function recursively splits the array into halves until reaching
    arrays of size one, which are inherently sorted. It then merges each
    sorted half while maintaining the order.

    Complexity:
    - Time: O(n log n) due to repeatedly dividing the array and merging the halves.
    - Space: O(n) because of the additional space used during the merge process.

    Parameters:
    arr (List[int]): The list of integers to be sorted.

    Returns:
    List[int]: A new sorted list of integers.
    """
    if len(arr) <= 1:
        # Base case: An array of zero or one elements is already sorted
        return arr

    # Split the array into two halves
    mid = len(arr) // 2
    l